In [33]:
import pandas as pd
import pybamm
import pybop

In [34]:
parameter_set = pybamm.ParameterValues("ECM_Example")
parameter_set.update(
    {
        "Cell capacity [A.h]": 5,
        "Nominal cell capacity [A.h]": 5,
        # "Current function [A]": 5,
        # "Initial SoC": 0.5,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.0,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 2e-4,
        "C1 [F]": 1e4,
        "Open-circuit voltage [V]": pybop.empirical.Thevenin().default_parameter_values[
            "Open-circuit voltage [V]"
        ],
    }
)
# Optional arguments - only needed for two RC pairs
parameter_set.update(
    {
        "R2 [Ohm]": 0.0003,
        "C2 [F]": 40000,
        "Element-2 initial overpotential [V]": 0,
    },
    check_already_exists=False,
)

In [35]:
model = pybop.empirical.Thevenin(
    parameter_set=parameter_set,
    options={"number of rc elements": 2},
    solver=pybamm.CasadiSolver(mode="safe", dt_max=10),
)

file_loc = r"../hppc_lut/G1/battery_G1_cycle_2_pulse_1_lut.csv"
df = pd.read_csv(file_loc, index_col=None, na_values=["NA"])
df = df.drop_duplicates(subset=["Time"], keep="first")

dataset = pybop.Dataset(
    {
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy(), 
        "Voltage [V]": df["Voltage"].to_numpy(),
    }
)

In [36]:
r_guess = 0.005
parameters = pybop.Parameters(
    pybop.Parameter(
        "R0 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "R1 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "R2 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "C1 [F]",
        prior=pybop.Gaussian(500, 100),
        bounds=[1, 1500],
    ),
    pybop.Parameter(
        "C2 [F]",
        prior=pybop.Gaussian(2000, 500),
        bounds=[0, 25000],
    ),
)

In [39]:
initial_state_of_charge= df["SoC"].iloc[0] / 100 

model.build(
    initial_state={"Initial open-circuit voltage [V]": df["Voltage"].to_numpy()[0]}
    # initial_state={"Initial SoC": initial_state_of_charge} # soc scales between 0 to 1
)
problem = pybop.FittingProblem(
    model,
    parameters,
    dataset,
)

cost = pybop.SumSquaredError(problem)

In [40]:
optim = pybop.PSO(
    cost,
    sigma0=[1e-3, 1e-3, 1e-3, 50, 500],
    max_unchanged_iterations=30,
    max_iterations=100,
)
results = optim.run()

Halt: No significant change for 30 iterations.
OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [4.84657111e-03 5.36928072e-03 4.51946657e-03 5.20629563e+02
 2.18035131e+03]
  Optimised parameters: [2.84314465e-02 1.09936182e-02 6.44431560e-03 7.89020179e+02
 6.55349671e+03]
  Total-order sensitivities:
  Diagonal Fisher Information entries: None
  Final cost: 0.0873427576979139
  Optimisation time: 131.88694429397583 seconds
  Number of iterations: 76
  Number of evaluations: 475
  SciPy result available: No
  PyBaMM Solution available: Yes


In [41]:
pybop.plot.quick(problem, problem_inputs=results.x, title="Optimised Comparison");

In [42]:
pybop.plot.convergence(optim)
pybop.plot.parameters(optim);

In [32]:
import os
import pandas as pd
import pybamm
import pybop

# 298.15 K // 25 degree C
FIXED_TEMPERATURE = 298.15

# Prepare an empty list to store LUT data for each pulse
lut = []

# Loop over pulses 0 to 9
for pulse in range(10):
    file_loc = f"../hppc_lut/G1/battery_G1_cycle_2_pulse_{pulse}_lut.csv"
    print(f"Processing {file_loc}...")
    
    # Read and clean the CSV data
    df = pd.read_csv(file_loc, na_values=["NA"])
    df = df.drop_duplicates(subset=["Time"], keep="first")
    
    # Create the dataset for pybop
    dataset = pybop.Dataset({
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy(), 
        "Voltage [V]": df["Voltage"].to_numpy(),
    })
    
    # Update parameter set for the ECM
    parameter_set = pybamm.ParameterValues("ECM_Example")
    parameter_set.update({
        "Cell capacity [A.h]": 5,
        "Nominal cell capacity [A.h]": 5,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.0,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 2e-4,
        "C1 [F]": 1e4,
        "Open-circuit voltage [V]": pybop.empirical.Thevenin().default_parameter_values["Open-circuit voltage [V]"]
    })
    parameter_set.update({
        "R2 [Ohm]": 0.0003,
        "C2 [F]": 40000,
        "Element-2 initial overpotential [V]": 0,
    }, check_already_exists=False)
    
    # Build the ECM model
    model = pybop.empirical.Thevenin(
        parameter_set=parameter_set,
        options={"number of rc elements": 2},
        solver=pybamm.CasadiSolver(mode="safe", dt_max=10),
    )
    
    # Extract the initial SoC from the CSV file (scaled from 0 to 1)
    initial_soc = df["SoC"].iloc[0] / 100
    #model.build(initial_state={"Initial SoC": initial_soc})
    model.build(
        initial_state={"Initial open-circuit voltage [V]": df["Voltage"].to_numpy()[0]}
        # initial_state={"Initial SoC": initial_soc} # soc scales between 0 to 1
    )

    # Set up the parameters for the optimization
    r_guess = 0.005
    parameters = pybop.Parameters(
        pybop.Parameter("R0 [Ohm]", prior=pybop.Gaussian(r_guess, r_guess / 10), bounds=[0, 0.2]),
        pybop.Parameter("R1 [Ohm]", prior=pybop.Gaussian(r_guess, r_guess / 10), bounds=[0, 0.2]),
        pybop.Parameter("R2 [Ohm]", prior=pybop.Gaussian(r_guess, r_guess / 10), bounds=[0, 0.2]),
        pybop.Parameter("C1 [F]", prior=pybop.Gaussian(500, 100), bounds=[1, 1500]),
        pybop.Parameter("C2 [F]", prior=pybop.Gaussian(2000, 500), bounds=[0, 25000]),
    )
    
    # Define and solve the fitting problem
    problem = pybop.FittingProblem(model, parameters, dataset)
    cost = pybop.SumSquaredError(problem)
    optim = pybop.PSO(
        cost,
        sigma0=[1e-3, 1e-3, 1e-3, 50, 500],
        max_unchanged_iterations=30,
        max_iterations=100,
    )
    results = optim.run()
    
    # The results vector (results.x) -- contains the optimized parameters in order to "parameters":
    # R0, R1, R2, C1, C2. We are interested in R0, R1, and C1.
    r0, r1, r2, c1, c2 = results.x
    
    # Create a LUT entry for the current pulse
    pulse_entry = {
        #"pulse": pulse,
        "current": df["Current"][1],
        "voltage": df["Voltage"][0],
        "temperature": FIXED_TEMPERATURE,
        "r0": r0,
        "r1": r1,
        "c1": c1,
        "SoC": df["SoC"][0]
    }
    lut.append(pulse_entry)

# Optionally, convert the LUT list to a DataFrame and save as CSV
lut_df = pd.DataFrame(lut)
lut_df.to_csv("ecm_lut_table.csv", index=False)
print("LUT table saved to ecm_lut_table.csv")


Processing ../hppc_lut/G1/battery_G1_cycle_2_pulse_0_lut.csv...
Halt: No significant change for 30 iterations.
OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [5.63241247e-03 4.87035035e-03 5.04957550e-03 5.53382784e+02
 2.32799986e+03]
  Optimised parameters: [3.28974797e-02 1.18519478e-02 9.35261623e-04 8.01187350e+02
 1.88730188e+02]
  Total-order sensitivities:
  Diagonal Fisher Information entries: None
  Final cost: 0.6463261679032071
  Optimisation time: 136.63282704353333 seconds
  Number of iterations: 79
  Number of evaluations: 495
  SciPy result available: No
  PyBaMM Solution available: Yes
Processing ../hppc_lut/G1/battery_G1_cycle_2_pulse_1_lut.csv...
Halt: No significant change for 30 iterations.
OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [4.83134095e-03 4.75557399e-03 4.39733908e-03 4.95995934e+02
 2.04607618e+03]
  Optimised parameters: [2.17733186e-02 1.30903146e-02 1.04942256e-02 3.40354949e+01
 2.60522685e+03]
  To

ValueError: Optimised parameters do not produce a finite cost value

In [10]:
import pandas as pd

# Define the input and output file paths
input_file = 'G1_cycle2_ecm_lut_table.csv'  # Replace with your input file path
output_dir = 'data/'  # Directory where the output files will be saved

# Read the input CSV file
data = pd.read_csv(input_file)

# Extract relevant columns (r0, r1, c1, and SoC)
r0_data = data[['temperature', 'current', 'SoC', 'r0']]
r1_data = data[['temperature', 'current', 'SoC', 'r1']]
c1_data = data[['temperature', 'current', 'SoC', 'c1']]

# Save each to a new CSV file
r0_data.to_csv(f'{output_dir}r0.csv', index=False)
r1_data.to_csv(f'{output_dir}r1.csv', index=False)
c1_data.to_csv(f'{output_dir}c1.csv', index=False)

print(f"Files have been saved to {output_dir}")

Files have been saved to data/


In [31]:
import pybamm
import os
import numpy as np
import matplotlib.pyplot as plt

# Load your data (you've already done this part)
path = os.getcwd()
ocv_data = pybamm.parameters.process_1D_data("data\\G1_ocv.csv", path=path)
r0_data = pybamm.parameters.process_2D_data_csv("data\\r0.csv", path=path)
r1_data = pybamm.parameters.process_2D_data_csv("data\\r1.csv", path=path)
c1_data = pybamm.parameters.process_2D_data_csv("data\\c1.csv", path=path)

# Interpolation functions (you've already done this part)
def ocv(sto):
    name, (x, y) = ocv_data
    return pybamm.Interpolant(x, y, sto, name)

def r0(T_cell, current, soc):
    name, (x, y) = r0_data
    return pybamm.Interpolant(x, y, [T_cell, current, soc], name)

def r1(T_cell, current, soc):
    name, (x, y) = r1_data
    return pybamm.Interpolant(x, y, [T_cell, current, soc], name)

def c1(T_cell, current, soc):
    name, (x, y) = c1_data
    return pybamm.Interpolant(x, y, [T_cell, current, soc], name)

# Parameter values function (you've already started this)
def get_parameter_values():
    # Your parameter values as defined
    cell_capacity = 100
    
    values = {
        "chemistry": "ecm",
        "Initial SoC": 0.5,
        "Initial temperature [K]": 25 + 273.15,
        "Cell capacity [A.h]": cell_capacity,
        "Nominal cell capacity [A.h]": cell_capacity,
        "Ambient temperature [K]": 25 + 273.15,
        "Current function [A]": 100,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.2,
        "Cell thermal mass [J/K]": 1000,
        "Cell-jig heat transfer coefficient [W/K]": 10,
        "Jig thermal mass [J/K]": 500,
        "Jig-air heat transfer coefficient [W/K]": 10,
        "Open-circuit voltage [V]": ocv,
        "R0 [Ohm]": r0,
        "Element-1 initial overpotential [V]": 0,
        "R1 [Ohm]": r1,
        "C1 [F]": c1,
        "RCR lookup limit [A]": 340,
    }
    
    return values


model = pybamm.equivalent_circuit.Thevenin(options={"number of rc elements":1})
parameter_values = pybamm.ParameterValues(values=get_parameter_values())

t_eval = np.linspace(0, 3600, 100)
solver = pybamm.CasadiSolver(mode="safe", atol=1e-6, rtol=1e-3)

sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve(t_eval=t_eval)
sim.plot()


ValueError: len(x) should equal len(children)